In [1]:
# This notebook is only for preprocessing the data which contains:
    # 1. Importing the dataframes sourced from SQL
    # 2. Pivotting the LABEL table's values with label_value and maks_probability columns
    # 3. Merging the clip_encode data with pivot table created on the previous step
    # 4. Dropping unwanted columns such as ID columns & VMAF
    # 5. Train and Test data split with 20%
    # 6. Converting categorical columns to numericals
    # 7. Exporting Train and Test datasets
    # /// please use exported datasets to build models ///

In [2]:
# Import Libraries
import pandas as pd
from sklearn import preprocessing
from joblib import dump, load
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
# Load clip & encode dataset
df_clip_encode = pd.read_csv("../clip_encode_merged_data.csv", encoding = 'utf8')
df_clip_encode.head(1)

,encode_id,clip_id,encode_width,encode_height,clip_width,clip_height,clip_duration,clip_size,clip_bitrate_total,video_profile,clip_frame_rate,crf,encode_bitrate_video,nr_of_images,nr_of_image_shifts,psnr,vmaf
0,7,5,1920,1080,1920,1080,19,41249088,17350686,High 4:2:2,30000/1001,18,18667784.0,3,8,44.62155,99.406406


In [11]:
# Load Label dataset
df_label = pd.read_csv("../label_data.csv", encoding = 'utf8')
df_label.head(5)

,label_clip_id,label_value,maks_probability
0,5,"chime, bell, gong",57.84
1,5,"spider web, spider's web",68.54
2,5,"flagpole, flagstaff",60.99
3,8,"mobile home, manufactured home",56.77
4,6,"African chameleon, Chamaeleo chamaeleon",81.73


## Pivotting Label dataset

In [10]:
df_label_pivot = pd.pivot_table(df_label, values='maks_probability', index=['label_clip_id'],
                     columns=['label_value'], aggfunc=np.sum, fill_value = 0)
df_label_pivot.head(5)

label_value,"African chameleon, Chamaeleo chamaeleon","American egret, great white heron, Egretta albus",Dutch oven,abaya,alp,altar,analog clock,"balance beam, beam",balloon,"ballplayer, baseball player",...,trench coat,turnstile,"vacuum, vacuum cleaner","wardrobe, closet, press","web site, website, internet site, site",wig,window screen,window shade,wine bottle,"worm fence, snake fence, snake-rail fence, Virginia fence"
label_clip_id,,,,,,,,,,,,,,,,,,,,,
5,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
6,81.73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
7,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,56.98,0.0,0.0,0.0
8,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
9,73.35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


## Merge datasets: clip & encode with label on clip_id

In [6]:
df_merged = df_clip_encode.merge(df_label_pivot, how = 'inner', left_on='clip_id', right_on='label_clip_id')

In [7]:
# drop unwanted columns
df_merged = df_merged.drop(columns=['encode_id', 'clip_id', 'psnr'], axis=1)
#df_merged = df_merged.drop(columns=['encode_id', 'clip_id', 'psnr', 'nr_of_images', 'nr_of_image_shifts'], axis=1)

In [8]:
#drop NULL rows
#here I simply dropped the null values and ofcourse lost some info. Later I will try to ampute data with mean.
print(df_merged.shape)
df_merged = df_merged.dropna()
print(df_merged.shape)

(5479, 151)
(4790, 151)


In [9]:
df_merged.head(5)

,encode_width,encode_height,clip_width,clip_height,clip_duration,clip_size,clip_bitrate_total,video_profile,clip_frame_rate,crf,...,trench coat,turnstile,"vacuum, vacuum cleaner","wardrobe, closet, press","web site, website, internet site, site",wig,window screen,window shade,wine bottle,"worm fence, snake fence, snake-rail fence, Virginia fence"
0,1920,1080,1920,1080,19,41249088,17350686,High 4:2:2,30000/1001,18,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1920,1080,1920,1080,19,41249088,17350686,High 4:2:2,30000/1001,19,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1920,1080,1920,1080,19,41249088,17350686,High 4:2:2,30000/1001,20,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1920,1080,1920,1080,19,41249088,17350686,High 4:2:2,30000/1001,27,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1920,1080,1920,1080,19,41249088,17350686,High 4:2:2,30000/1001,29,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Test & Train Split

In [69]:
df_train, df_test = train_test_split(df_merged, test_size=0.2)
print(df_train.shape, df_test.shape)

(3832, 151) (958, 151)


## Label Binarizer: Converting set of categorical columns to numeric

In [70]:
categorical_columns =['encode_width', 'encode_height', 'clip_width', 'clip_height','video_profile', 'clip_frame_rate'] 

for column in categorical_columns:
    lb = preprocessing.LabelBinarizer()
    lb.fit(df_train[column].values.reshape(-1,1))
    dump(lb, f"{column}_label_binarizer.pkl")
    binarized_array_train = lb.transform(df_train[column].values.reshape(-1,1))
    column_names = [f"{column}_{i}" for i in list(lb.classes_)]
    
    binarized_df_train = pd.DataFrame(data=binarized_array_train, columns=column_names, index=df_train.index)
    df_train = pd.concat([df_train, binarized_df_train], axis=1, sort=False)
    df_train.drop(columns=[column], inplace=True)

    binarized_array_test = lb.transform(df_test[column].values.reshape(-1,1))
    binarized_df_test = pd.DataFrame(data=binarized_array_test, columns=column_names, index=df_test.index)
    df_test = pd.concat([df_test, binarized_df_test], axis=1, sort=False)
    df_test.drop(columns=[column], inplace=True)
    print(column + ' is converted')
    print(df_train.shape, df_test.shape)

encode_width is converted
(3832, 157) (958, 157)
encode_height is converted
(3832, 162) (958, 162)
clip_width is converted
(3832, 162) (958, 162)
clip_height is converted
(3832, 165) (958, 165)
video_profile is converted
(3832, 169) (958, 169)
clip_frame_rate is converted
(3832, 174) (958, 174)


## Export datasets: Test & Train

In [72]:
export_df_train = df_train.to_csv (r'export_df_train.csv', index = None, header=True) 

In [73]:
export_df_test = df_test.to_csv (r'export_df_test.csv', index = None, header=True) 

### / please use exported datasets to build models /